# Pipeline Project

You will be using the provided data to create a machine learning model pipeline.

You must handle the data appropriately in your pipeline to predict whether an
item is recommended by a customer based on their review.
Note the data includes numerical, categorical, and text data.

You should ensure you properly train and evaluate your model.

## The Data

The dataset has been anonymized and cleaned of missing values.

There are 8 features for to use to predict whether a customer recommends or does
not recommend a product.
The `Recommended IND` column gives whether a customer recommends the product
where `1` is recommended and a `0` is not recommended.
This is your model's target/

The features can be summarized as the following:

- **Clothing ID**: Integer Categorical variable that refers to the specific piece being reviewed.
- **Age**: Positive Integer variable of the reviewers age.
- **Title**: String variable for the title of the review.
- **Review Text**: String variable for the review body.
- **Positive Feedback Count**: Positive Integer documenting the number of other customers who found this review positive.
- **Division Name**: Categorical name of the product high level division.
- **Department Name**: Categorical name of the product department name.
- **Class Name**: Categorical name of the product class name.

The target:
- **Recommended IND**: Binary variable stating where the customer recommends the product where 1 is recommended, 0 is not recommended.

## Load Data

In [1]:
import pandas as pd

# Load data
df = pd.read_csv(
    'data/reviews.csv',
)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18442 entries, 0 to 18441
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              18442 non-null  int64 
 1   Age                      18442 non-null  int64 
 2   Title                    18442 non-null  object
 3   Review Text              18442 non-null  object
 4   Positive Feedback Count  18442 non-null  int64 
 5   Division Name            18442 non-null  object
 6   Department Name          18442 non-null  object
 7   Class Name               18442 non-null  object
 8   Recommended IND          18442 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.3+ MB


,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name,Recommended IND
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses,0
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants,1
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses,1
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses,0
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits,1


## Preparing features (`X`) & target (`y`)

In [2]:
data = df

# separate features from labels
X = data.drop('Recommended IND', axis=1)
y = data['Recommended IND'].copy()

print('Labels:', y.unique())
print('Features:')
display(X.head())

Labels: [0 1]
Features:


,Clothing ID,Age,Title,Review Text,Positive Feedback Count,Division Name,Department Name,Class Name
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,0,General,Dresses,Dresses
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",0,General Petite,Bottoms,Pants
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,6,General,Tops,Blouses
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",4,General,Dresses,Dresses
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,General Petite,Tops,Knits


In [3]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.1,
    shuffle=True,
    random_state=27,
)

# Your Work

## Data Exploration

In [4]:
#Check data shapes

print(f'X training data shape: {X_train.shape}')
print(f'y training data shape: {y_train.shape}')
print(f'X test data shape: {X_test.shape}')
print(f'y test data shape: {y_test.shape}')

X training data shape: (16597, 8)
y training data shape: (16597,)
X test data shape: (1845, 8)
y test data shape: (1845,)


In [5]:
# Check data overall for null values

nan_counts = data.isna().sum()
nan_counts

Clothing ID                0
Age                        0
Title                      0
Review Text                0
Positive Feedback Count    0
Division Name              0
Department Name            0
Class Name                 0
Recommended IND            0
dtype: int64

In [6]:
# Get numerical descriptive statistics

X_train.describe()

,Clothing ID,Age,Positive Feedback Count
count,16597.000000,16597.000000,16597.000000
mean,954.951618,43.370609,2.713924
std,141.863331,12.216240,6.014332
min,2.000000,18.000000,0.000000
25%,863.000000,34.000000,0.000000
50%,952.000000,42.000000,1.000000
75%,1078.000000,52.000000,3.000000
max,1205.000000,99.000000,122.000000


In [7]:
# Get object descriptive statistics

X_train.describe(include = ['object'])

,Title,Review Text,Division Name,Department Name,Class Name
count,16597,16597,16597,16597,16597
unique,11955,16594,2,6,14
top,Love it!,I bought this shirt at the store and after goi...,General,Tops,Dresses
freq,120,2,10497,7818,4845


In [13]:
#Checking numerical features for outliers

def feature_outlier_count(df):

    q_low = df.quantile(0.05)
    q_high = df.quantile(0.95)
    cleaned_data = df[(df >= q_low) & (df <= q_high)]
    return df.shape[0] - cleaned_data.shape[0]

for feature in X_train.columns:
    if X_train[feature].dtype.kind in 'iu':  # i = signed int, u = unsigned int
        print(f"{feature} outlier count is: {feature_outlier_count(X_train[feature])}")
                    

Clothing ID outlier count is: 1653
Age outlier count is: 1646
Positive Feedback Count outlier count is: 782


## Building Pipeline

In [9]:
from sklearn.pipeline import Pipeline


In [10]:
#Split features into numerical, categorical and text

num_data = X_train[['Clothing ID', 'Age', 'Positive Feedback Count']].copy()
cat_data = X_train[['Division Name', 'Department Name', 'Class Name']].copy()
txt_data = X_train[['Title', 'Review Text']].copy()

### Build Numerical Pipeline

In [ ]:
from sklearn.

### Build Categorical Pipeline

### Build Text Pipeline

## Training Pipeline

## Fine-Tuning Pipeline